In [ ]:
## Module import
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from scipy import stats
import seaborn as sns
import cartopy.feature as cft
import matplotlib.path as mpath
import cmocean as cm
import string
import math
from datetime import datetime, timedelta
import proplot as pplt
import xesmf as xe
import netCDF4 as nc

## 1) Standard deviation

In [ ]:
# open IPSL file
netCDF_T95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_T_for_std_19952014.nc'
TEMP_CMIP95 = xr.open_mfdataset(netCDF_T95_file)
TEMP_CMIP95

netCDF_S95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_S_for_std_19952014.nc'
SAL_CMIP95 = xr.open_mfdataset(netCDF_S95_file)
SAL_CMIP95

# open MPI file
netCDF_MPI_T95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_T_for_std_19952014.nc'
TEMP_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_T95_file)
TEMP_MPI_CMIP95

netCDF_MPI_S95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_S_for_std_19952014.nc'
SAL_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_S95_file)
SAL_MPI_CMIP95

# open UKESM file
netCDF_UKESM_T95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_T_for_std_19952014.nc'
TEMP_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_T95_file)
TEMP_UKESM_CMIP95

netCDF_UKESM_S95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_S_for_std_19952014.nc'
SAL_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_S95_file)
SAL_UKESM_CMIP95

# open topo
netCDF_topom_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/deptho_Ofx_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn.nc'
TOPOM = xr.open_mfdataset(netCDF_topom_file)

netCDF_topou_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/deptho_Ofx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc'
TOPOU = xr.open_mfdataset(netCDF_topou_file)

# correction topo pour contour
delta_lon = np.abs(np.diff(TOPOM['longitude']))
for i, start in enumerate(np.argmax(delta_lon > 180, axis=1)):
    TOPOM['longitude'][i, start+1:] += 360
    
delta_lon1 = np.abs(np.diff(TOPOU['longitude']))
for i, start in enumerate(np.argmax(delta_lon1 > 180, axis=1)):
    TOPOU['longitude'][i, start+1:] += 360
    
# open area file
netCDF_area = '/Users/jcaillet/Documents/Elmer/DATA/BATHY/BedMachineAntarctica_v02_m.nc'
Area = xr.open_mfdataset(netCDF_area)

## 2) T and S Profils

### 2.1) Observations

#### 2.1.1) WOA18

In [1]:
############################################ Open files ##########################################################

# Temperature 1995-2017
netCDF_fileT0 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/T/woa18_1995-2004_t00_01.nc'
netCDF_fileT1 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/T/woa18_2005-2017_t00_01.nc'
OBST095 = xr.open_mfdataset(netCDF_fileT0, decode_cf=False)
OBST195 = xr.open_mfdataset(netCDF_fileT1, decode_cf=False)
OBST95 = xr.merge([OBST095,OBST195])
OBST95 = OBST95.mean('time')

# Salinity 1995-2017
netCDF_fileS0 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/S/woa18_1995-2004_s00_01.nc'
netCDF_fileS1 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/S/woa18_2005-2017_s00_01.nc'
OBSS095 = xr.open_mfdataset(netCDF_fileS0, decode_cf=False)
OBSS195 = xr.open_mfdataset(netCDF_fileS1, decode_cf=False)
OBSS95 = xr.merge([OBSS095,OBSS195])
OBSS95 = OBSS95.mean('time')

# mask file of Amundsen Sea
netCDF_fileT = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/00_mask_continental_shelf.nc'
Mask = xr.open_mfdataset(netCDF_fileT)

# grid file of Amundsen Sea
netCDF_grid_file = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/mesh_mask_AMUXL025_BedMachineAntarctica-2020-07-15_v02_ICB380.nc'
Grid = xr.open_mfdataset(netCDF_grid_file).squeeze()

##################################  Interpolation of Amundsen sea mask on grid 1°x1° ##############################

# add lon, lat to mask file
Grid['Mask'] = Grid['mbathy'] - Grid['mbathy'] + Mask.isfmask.values
Grid = Grid.rename({"nav_lon": "lon", "nav_lat": "lat"})

# interpolation on grid 1°x1° (same as Observation)
## target Grid
ds_out = xe.util.grid_global(1, 1)
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, ds_out, "bilinear")
## regridding
dr_out = regridder(dr)
## rename
dr_out = dr_out.rename({"lon": "nav_lon", "lat": "nav_lat"})
dr_out = dr_out.rename({"x": "lon", "y": "lat"})

# add to data
OBST95['mask'] = dr_out
OBSS95['mask'] = dr_out

###################################### Mean profil on the continental shelf #######################################

# non-value : np.nan instead of 1e36
OBST95['temp']= xr.where(OBST95.t_an>1000,np.nan,OBST95.t_an.values)
OBST95['depth'] = - OBST95['depth']

OBSS95['sal'] = xr.where(OBSS95.s_an>1000,np.nan,OBSS95.s_an.values)
OBSS95['depth'] = - OBSS95['depth']

# mean T profil on continental shelf
OBST95_Amundsen = OBST95.temp.where(OBST95.mask==1).mean(['lon','lat']) ## approximation cells have the same size

# mean S profil on continental shelf
OBSS95_Amundsen = OBSS95.sal.where(OBSS95.mask==1).mean(['lon','lat']) ## approximation cells have the same size

NameError: name 'xr' is not defined

#### 2.1.2) ISMIP6 climatology

In [ ]:
########################################### Open files ##########################################################

# salinity
netCDF_file_sal = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/S/obs_salinity_1995-2017_8km_x_60m.nc'
OBSSC = xr.open_mfdataset(netCDF_file_sal, decode_cf=False)
# temperature
netCDF_file_temp = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/T/obs_temperature_1995-2017_8km_x_60m.nc'
OBSTC = xr.open_mfdataset(netCDF_file_temp, decode_cf=False)

# mask of BedMachinev2
netCDF_front_file = '/Users/jcaillet/Documents/Elmer/DATA/BATHY/BedMachineAntarctica_v02_m.nc'
bed = xr.open_mfdataset(netCDF_front_file, decode_cf=False)

############################################## Formatting ##########################################################

# remove values on the land and beyond the bed elevation (using mask of betMachinev2)
bed_interp = bed.interp_like(OBSTC) # interpolation on the observed grid
OBSSC['salinity'] = OBSSC.salinity.where(bed_interp.mask==0)
OBSTC['temperature'] = OBSTC.temperature.where(bed_interp.mask==0)
OBSTC['bed'] = OBSTC['lon'] - OBSTC['lon'] + bed_interp.bed.values
OBSSC['bed'] = OBSTC['lon'] - OBSTC['lon'] + bed_interp.bed.values

# interpolation of Amundsen mask on the ISMIP6 grid
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, OBSSC, "bilinear")
## regridding
dr_out = regridder(dr)
# add to data
OBSTC['mask'] = OBSTC['lon'] - OBSTC['lon'] + dr_out.values
OBSSC['mask'] = OBSSC['lon'] - OBSSC['lon'] + dr_out.values

# apply mask of bed elevation (temperature set to nan if depth > bed elevation)
OBSTC['temp_corr'] = xr.where(OBSTC['bed'] < OBSTC['z'], OBSTC['temperature'],np.nan)
OBSSC['sal_corr'] = xr.where(OBSSC['bed'] < OBSSC['z'], OBSSC['salinity'],np.nan)

# vertical profil (as the cells have the same area, we can take the mean)
OBST_mean = OBSTC['temp_corr'].where(OBSTC['mask']==1).mean(['x','y'])
OBSS_mean = OBSSC['sal_corr'].where(OBSSC['mask']==1).mean(['x','y'])

### 2.2) CMIP6 outputs

#### 2.2.1) IPSL-CM6A-LR model

In [ ]:
# Temperature & salinty 1995-2014
netCDF = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/profil_T95_IPSL.nc'
CMIPT95I = xr.open_mfdataset(netCDF,concat_dim='members',combine='nested')
CMIPT95I = CMIPT95I.sortby('members')

netCDFS = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/profil_S95_IPSL.nc'
CMIPS95I = xr.open_mfdataset(netCDFS,concat_dim='members',combine='nested')
CMIPS95I = CMIPS95I.sortby('members')

# Area of cells
netCDF_area = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/areacello_Ofx_IPSL-CM6A-LR_historical_r1i1p1f1_gn.nc'
Area = xr.open_mfdataset(netCDF_area).sel(y=slice(0,100), x=slice(100,250))

# mask file of Amundsen Sea
netCDF_fileT = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/00_mask_continental_shelf.nc'
Mask = xr.open_mfdataset(netCDF_fileT)

# grid file of Amundsen Sea
netCDF_grid_file = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/mesh_mask_AMUXL025_BedMachineAntarctica-2020-07-15_v02_ICB380.nc'
Grid = xr.open_mfdataset(netCDF_grid_file).squeeze()

# add lon, lat to mask file
Grid['Mask'] = Grid['mbathy'] - Grid['mbathy'] + Mask.isfmask.values
Grid = Grid.rename({"nav_lon": "lon", "nav_lat": "lat"})

# Interpolation of Amundsen Mask on the CMIP6 Grid
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, CMIPT95I, "bilinear")
## regridding
dr_out = regridder(dr)
## add to data
CMIPT95I['mask'] = CMIPT95I['nav_lon'] - CMIPT95I['nav_lon'] + dr_out.values

CMIPT95I['area'] = Area['areacello'].broadcast_like(CMIPT95I['thetao'])
CMIPT95I['area'] = CMIPT95I['area'] * CMIPT95I['thetao'] / CMIPT95I['thetao']

# vertical profil
CMIPT95I_mean = (CMIPT95I['thetao'] * CMIPT95I['area']).where(CMIPT95I['mask']==1).sum(['x','y'])/ CMIPT95I['area'].where(CMIPT95I['mask']==1).sum(['x','y'])
CMIPT95I_mean['olevel']= - CMIPT95I_mean['olevel']

CMIPS95I_mean = (CMIPS95I['so'] * CMIPT95I['area']).where(CMIPT95I['mask']==1).sum(['x','y'])/ CMIPT95I['area'].where(CMIPT95I['mask']==1).sum(['x','y'])
CMIPS95I_mean['olevel']= - CMIPS95I_mean['olevel']

# mean members
CMIPT95I_mean_tot = CMIPT95I_mean.mean('members')

CMIPS95I_mean_tot = CMIPS95I_mean.mean('members')

#### 2.2.2) MPI-ESM1.2-HR model

In [ ]:
# Temperature & salinity 1995-2014
netCDF = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/profil_T95_MPI.nc'
CMIPT95M = xr.open_mfdataset(netCDF,concat_dim='members',combine='nested')
CMIPT95M = CMIPT95M.sortby('members')

netCDFS = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/profil_S95_MPI.nc'
CMIPS95M = xr.open_mfdataset(netCDFS,concat_dim='members',combine='nested')
CMIPS95M = CMIPS95M.sortby('members')

# Area of cells
netCDF_area = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/areacello_Ofx_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn.nc'
Area = xr.open_mfdataset(netCDF_area).sel(j=slice(380,400), i=slice(300,410))

# mask file of Amundsen Sea
netCDF_fileT = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/00_mask_continental_shelf.nc'
Mask = xr.open_mfdataset(netCDF_fileT)

# grid file of Amundsen Sea
netCDF_grid_file = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/mesh_mask_AMUXL025_BedMachineAntarctica-2020-07-15_v02_ICB380.nc'
Grid = xr.open_mfdataset(netCDF_grid_file).squeeze()

# add lon, lat to mask file
Grid['Mask'] = Grid['mbathy'] - Grid['mbathy'] + Mask.isfmask.values
Grid = Grid.rename({"nav_lon": "lon", "nav_lat": "lat"})

# Interpolation of Amundsen Mask on the CMIP6 Grid
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, CMIPT95M, "bilinear")
## regridding
dr_out = regridder(dr)
## add to data
CMIPT95M['mask'] = CMIPT95M['longitude'] - CMIPT95M['longitude'] + dr_out.values

CMIPT95M['area'] = Area['areacello'].broadcast_like(CMIPT95M['thetao'])
CMIPT95M['area'] = CMIPT95M['area'] * CMIPT95M['thetao'] / CMIPT95M['thetao']

# vertical profil
CMIPT95M_mean = (CMIPT95M['thetao'] * CMIPT95M['area']).where(CMIPT95M['mask']==1).sum(['i','j'])/ CMIPT95M['area'].where(CMIPT95M['mask']==1).sum(['i','j'])
CMIPT95M_mean['lev']= - CMIPT95M_mean['lev']

CMIPS95M_mean = (CMIPS95M['so'] * CMIPT95M['area']).where(CMIPT95M['mask']==1).sum(['i','j'])/ CMIPT95M['area'].where(CMIPT95M['mask']==1).sum(['i','j'])
CMIPS95M_mean['lev']= - CMIPS95M_mean['lev']

# mean members
CMIPT95M_mean_tot = CMIPT95M_mean.mean('members')

CMIPS95M_mean_tot = CMIPS95M_mean.mean('members')

#### 2.2.3) UKESM1-0-LL model

In [ ]:
# Temperature & salinity 1995-2014
netCDF = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/profil_T95_UKESM.nc'
CMIPT95U = xr.open_mfdataset(netCDF,concat_dim='members',combine='nested')
CMIPT95U = CMIPT95U.sortby('members')

netCDFS = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/profil_S95_UKESM.nc'
CMIPS95U = xr.open_mfdataset(netCDFS,concat_dim='members',combine='nested')
CMIPS95U = CMIPS95U.sortby('members')

# Area of cells
netCDF_area = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/areacello_Ofx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc'
Area = xr.open_mfdataset(netCDF_area).sel(j=slice(30,80), i=slice(130,200))

# mask file of Amundsen Sea
netCDF_fileT = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/00_mask_continental_shelf.nc'
Mask = xr.open_mfdataset(netCDF_fileT)

# grid file of Amundsen Sea
netCDF_grid_file = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/mesh_mask_AMUXL025_BedMachineAntarctica-2020-07-15_v02_ICB380.nc'
Grid = xr.open_mfdataset(netCDF_grid_file).squeeze()

# add lon, lat to mask file
Grid['Mask'] = Grid['mbathy'] - Grid['mbathy'] + Mask.isfmask.values
Grid = Grid.rename({"nav_lon": "lon", "nav_lat": "lat"})

# Interpolation of Amundsen Mask on the CMIP6 Grid
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, CMIPT95U, "bilinear")
## regridding
dr_out = regridder(dr)
## add to data
CMIPT95U['mask'] = CMIPT95U['longitude'] - CMIPT95U['longitude'] + dr_out.values

CMIPT95U['area'] = Area['areacello'].broadcast_like(CMIPT95U['thetao'])
CMIPT95U['area'] = CMIPT95U['area'] * CMIPT95U['thetao'] / CMIPT95U['thetao']

# vertical profil
CMIPT95U_mean = (CMIPT95U['thetao'] * CMIPT95U['area']).where(CMIPT95U['mask']==1).sum(['i','j'])/ CMIPT95U['area'].where(CMIPT95U['mask']==1).sum(['i','j'])
CMIPT95U_mean['lev']= - CMIPT95U_mean['lev']

CMIPS95U_mean = (CMIPS95U['so'] * CMIPT95U['area']).where(CMIPT95U['mask']==1).sum(['i','j'])/ CMIPT95U['area'].where(CMIPT95U['mask']==1).sum(['i','j'])
CMIPS95U_mean['lev']= - CMIPS95U_mean['lev']

# mean members
CMIPT95U_mean_tot = CMIPT95U_mean.mean('members')

CMIPS95U_mean_tot = CMIPS95U_mean.mean('members')

### 3) Figure

In [ ]:
sns.set_context('paper')

I = np.arange(0,33,1)
M = np.arange(0,10,1)
U = np.arange(0,16,1)

land_50m = cft.NaturalEarthFeature('physical', 'land', '50m', edgecolor='black', facecolor='papayawhip', linewidth=0.5)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

fig, axs = plt.subplots(nrows = 4, ncols = 3, subplot_kw={'projection': ccrs.SouthPolarStereo()}, figsize = (7, 9), constrained_layout=True)
axs = axs.flatten()

for n, ax in enumerate(axs):
    ax.text(0.025, 0.9, string.ascii_lowercase[n]+')', transform=ax.transAxes, color='white', fontweight="bold")

for ax in np.array([axs[0],axs[1], axs[2],axs[3],axs[4],axs[5]]):
    ax.set_extent([-180, 180, -90, -63], crs=ccrs.PlateCarree())
    ax.add_feature(land_50m, color=[0.8, 0.8, 0.8])
    ax.coastlines(resolution='50m')
    gl0 = ax.gridlines(draw_labels=False, linewidth=0.75, color='gray', alpha=0.7, linestyle='--', dms=True, x_inline=False, y_inline=False)
    gl0.top_labels = False
    gl0.right_labels = False
    gl0.xlocator = mticker.FixedLocator([-180, -135, -90, -45, 0, 45, 90, 135, 180])
    gl0.ylocator = mticker.FixedLocator([-90, -80, -70, -60])
    gl0.xformatter = LongitudeFormatter()
    gl0.yformatter = LatitudeFormatter()
    gl0.xlabel_style = {'size': 12, 'color': 'gray'}
    gl0.ylabel_style = {'size': 12, 'color': 'gray'}
    #ax.set_boundary(circle, transform=ax.transAxes)


# Salinity 1995-2014
p4 = axs[0].pcolormesh(SAL_MPI_CMIP95.longitude, SAL_MPI_CMIP95.latitude, SAL_MPI_CMIP95['Sal_std_200700'],  cmap=cm.cm.haline,  vmin=0, vmax=0.06, transform=ccrs.PlateCarree(), shading='auto')
p44 = axs[0].contour(TOPOM.longitude, TOPOM.latitude, TOPOM['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

p3 = axs[1].pcolormesh(SAL_CMIP95.nav_lon, SAL_CMIP95.nav_lat, SAL_CMIP95['Sal_std_200700'],  cmap=cm.cm.haline, vmin=0, vmax=0.06, transform=ccrs.PlateCarree(), shading='auto')
p33 = axs[1].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

p5 = axs[2].pcolormesh(SAL_UKESM_CMIP95.longitude, SAL_UKESM_CMIP95.latitude, SAL_UKESM_CMIP95['Sal_std_200700'],  cmap=cm.cm.haline,  vmin=0, vmax=0.06, transform=ccrs.PlateCarree(), shading='auto')
p55 = axs[2].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

# Temperature 1995-2014
p1 = axs[3].pcolormesh(TEMP_MPI_CMIP95.longitude, TEMP_MPI_CMIP95.latitude, TEMP_MPI_CMIP95['Temp_std_200700'], cmap=cm.cm.thermal, vmin=0, vmax=0.4, transform=ccrs.PlateCarree(), shading='auto')
p11 = axs[3].contour(TOPOM.longitude, TOPOM.latitude, TOPOM['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

p0 = axs[4].pcolormesh(TEMP_CMIP95.nav_lon, TEMP_CMIP95.nav_lat, TEMP_CMIP95['Temp_std_200700'], cmap=cm.cm.thermal, vmin=0, vmax=0.4, transform=ccrs.PlateCarree(), shading='auto')
p00 = axs[4].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

p2 = axs[5].pcolormesh(TEMP_UKESM_CMIP95.longitude, TEMP_UKESM_CMIP95.latitude, TEMP_UKESM_CMIP95['Temp_std_200700'], cmap=cm.cm.thermal, vmin=0, vmax=0.4, transform=ccrs.PlateCarree(), shading='auto')
p22 = axs[5].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

ax_cb = plt.axes([0.087, 0.55, 0.01, 0.15])
cb = plt.colorbar(p2, cax=ax_cb, orientation='vertical')
cb.ax.set_ylabel('Standard deviation (°C)');

ax_cb1 = plt.axes([0.085, 0.79, 0.01, 0.15])
cb1 = plt.colorbar(p5, cax=ax_cb1, orientation='vertical')
cb1.ax.set_ylabel('Standard deviation (g kg$^{-1}$)');
  
plt.figtext(0.01, 0.865, 'Across member standard', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.03, 0.865, 'deviation of the 1995-2014', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.05, 0.865, 'mean practical salinity', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.01, 0.625, 'Across member standard', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.03, 0.625, 'deviation of the 1995-2014', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.05, 0.625, 'mean potential temperature', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.28, 0.97, 'MPI', fontweight="bold")
plt.figtext(0.56, 0.97, 'IPSL', fontweight="bold")
plt.figtext(0.82, 0.97, 'UKESM', fontweight="bold")
plt.figtext(0.01, 0.385, 'Salinity over the', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.03, 0.385, 'Amundsen Sea', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.05, 0.385, 'continental shelf', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.01, 0.145, 'Temperature over the', horizontalalignment='left', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.03, 0.145, 'Amundsen Sea', horizontalalignment='left', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.05, 0.145, 'continental shelf', horizontalalignment='left', verticalalignment='center', rotation=90, fontweight="bold")


## Temperature 1995-2014 - MPI
ax6 = fig.add_subplot(4,3,10)
#WOA
ax6.plot(OBST95_Amundsen, OBST95_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1995-2017')
#ISMIP6
ax6.plot(OBST_mean, OBST_mean.z, color = 'black', linewidth = 1.5, label= 'ISMIP6:1995-2017')
#CMIP6
for k in M:
    if k == 0:
        ax6.plot(CMIPT95M_mean[k,:], CMIPT95M_mean.lev, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax6.plot(CMIPT95M_mean[k,:], CMIPT95M_mean.lev, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax6.plot(CMIPT95M_mean_tot, CMIPT95M_mean_tot.lev, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax6.text(0.9, 0.9, 'j)', transform=ax6.transAxes, fontweight="bold")
ax6.set_ylabel('Depth (m)')
ax6.set_xlim(-2,2)
ax6.set_ylim(-2000,0)
plt.grid(True)

## Temperature 1995-2014 - IPSL
ax7 = fig.add_subplot(4,3,11)
#WOA
ax7.plot(OBST95_Amundsen, OBST95_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1995-2017')
#ISMIP6
ax7.plot(OBST_mean, OBST_mean.z, color = 'black', linewidth = 1.5, label= 'ISMIP6:1995-2017')
#CMIP6
for k in I:
    if k == 0:
        ax7.plot(CMIPT95I_mean[k,:], CMIPT95I_mean.olevel, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax7.plot(CMIPT95I_mean[k,:], CMIPT95I_mean.olevel, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax7.plot(CMIPT95I_mean_tot, CMIPT95I_mean_tot.olevel, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax7.text(0.9, 0.9, 'k)', transform=ax7.transAxes, fontweight="bold")
ax7.set_xlabel('Potential Temperature (°C)')
plt.legend(loc = 'lower left', prop={"size":8})
ax7.set_xlim(-2,2)
ax7.set_ylim(-2000,0)
plt.grid(True)
ax7.axes.yaxis.set_ticklabels([])

## Temperature 1995-2014 - UKESM
ax8 = fig.add_subplot(4,3,12)
#WOA
ax8.plot(OBST95_Amundsen, OBST95_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1995-2017')
#ISMIP6
ax8.plot(OBST_mean, OBST_mean.z, color = 'black', linewidth = 1.5, label= 'ISMIP6:1995-2017')
#CMIP6
for k in U:
    if k == 0:
        ax8.plot(CMIPT95U_mean[k,:], CMIPT95U_mean.lev, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax8.plot(CMIPT95U_mean[k,:], CMIPT95U_mean.lev, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax8.plot(CMIPT95U_mean_tot, CMIPT95U_mean_tot.lev, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax8.text(0.9, 0.9, 'l)', transform=ax8.transAxes, fontweight="bold")
ax8.set_xlim(-2,2)
ax8.set_ylim(-2000,0)
plt.grid(True)
ax8.axes.yaxis.set_ticklabels([])


## Salinity 1995-2014 - MPI
ax9 = fig.add_subplot(4,3,7)
#WOA
ax9.plot(OBSS95_Amundsen, OBSS95_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1995-2017')
#ISMIP6
ax9.plot(OBSS_mean, OBSS_mean.z, color = 'black', linewidth = 1.5, label= 'ISMIP6:1995-2017')
#CMIP6
for k in M:
    if k == 0:
        ax9.plot(CMIPS95M_mean[k,:], CMIPS95M_mean.lev, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax9.plot(CMIPS95M_mean[k,:], CMIPS95M_mean.lev, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax9.plot(CMIPS95M_mean_tot, CMIPS95M_mean_tot.lev, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax9.text(0.9, 0.9, 'g)', transform=ax9.transAxes, fontweight="bold")
ax9.set_ylabel('Depth (m)')
ax9.set_xlim(33.5,34.8)
ax9.set_ylim(-2000,0)
plt.grid(True)

## Salinity 1995-2014 - IPSL
ax10 = fig.add_subplot(4,3,8)
#WOA
ax10.plot(OBSS95_Amundsen, OBSS95_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1995-2017')
#ISMIP6
ax10.plot(OBSS_mean, OBSS_mean.z, color = 'black', linewidth = 1.5, label= 'ISMIP6:1995-2017')
#CMIP6
for k in I:
    if k == 0:
        ax10.plot(CMIPS95I_mean[k,:], CMIPS95I_mean.olevel, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax10.plot(CMIPS95I_mean[k,:], CMIPS95I_mean.olevel, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax10.plot(CMIPS95I_mean_tot, CMIPS95I_mean_tot.olevel, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax10.text(0.9, 0.9, 'h)', transform=ax10.transAxes, fontweight="bold")
ax10.set_xlabel('Practical Salinity (g kg$^{-1}$)')
plt.legend(loc = 'lower left', prop={"size":8})
ax10.set_xlim(33.5,34.8)
ax10.set_ylim(-2000,0)
plt.grid(True)
ax10.axes.yaxis.set_ticklabels([])

## Salinity 1995-2014 - UKESM
ax11 = fig.add_subplot(4,3,9)
#WOA
ax11.plot(OBSS95_Amundsen, OBSS95_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1995-2017')
#ISMIP6
ax11.plot(OBSS_mean, OBSS_mean.z, color = 'black', linewidth = 1.5, label= 'ISMIP6:1995-2017')
#CMIP6
for k in U:
    if k == 0:
        ax11.plot(CMIPS95U_mean[k,:], CMIPS95U_mean.lev, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax11.plot(CMIPS95U_mean[k,:], CMIPS95U_mean.lev, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax11.plot(CMIPS95U_mean_tot, CMIPS95U_mean_tot.lev, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax11.text(0.9, 0.9, 'i)', transform=ax11.transAxes, fontweight="bold")
ax11.set_xlim(33.5,34.8)
ax11.set_ylim(-2000,0)
plt.grid(True)
ax11.axes.yaxis.set_ticklabels([])

plt.tight_layout(pad = 1.0,rect=[0.07, 0, 1, 0.97])

plt.savefig('/Users/jcaillet/Documents/Elmer/CMIP6/FIGURES/Figure2_std19952014_modif.png',dpi=300)